In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
import copy
import re
import os
from pathlib import Path
from itertools import chain
from functools import partial

import blpapi
# import clarion
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import DateOffset, BDay
from pandas.tseries.frequencies import to_offset
from plotly import graph_objects as go, express as px
from plotly.subplots import make_subplots
# from clarion import positions

from api.gui.base import View, Tabs, Store, store, router
from api.gui.utils import get_bond_ref, get_bday, get_history_table, get_issuers_curves, get_issuer_curve
from api.gui.market_monitor import CurvesViewer, Pivot
from api.gui.max_diff import LowestFieldCurve
from api.gui.visual import Visual
from api.gui.clarion_comp import Comp
from api.gui.cdx import CDXComp
from api.gui.scen_grid import Grid
from api.gui.market_monitor_h import CurvesViewerH, PivotH
from api.gui.max_diff_h import LowestFieldCurveH
from api.gui.visual_h import VisualH
from api.gui.fwd_pair_h import ForwardH
from api.gui.fwd_pivot_h import PivotForwardH
from api.gui.issuer_hazard_h import HazardH
from api.utils import parse_offset, today
from api.blpw import BlpQuery
from api.bbg import BBGField, BBGOverrides
from api.data.base import data_path, tables
from api.data.curves import curves
from api.data.utils import add_pcs, parse_tenor
from api.pricing.cds import get_isda_cds
from api.data.processing import (
    apply_function, 
    add_years_to_maturity, 
    add_maturity_bucket,
    add_custom_rating,
    add_rating_bucket,
    add_geo_area
)

In [3]:
dt_fmt = '%Y-%m-%d'
bbg_dt_fmt = '%Y%m%d'

In [4]:
bq = BlpQuery(timeout=50000).start()

In [5]:
br = tables.bonds_reference
bh = pd.read_csv(data_path / 'bonds_history_all.csv', parse_dates=['date'], dayfirst=True)
cds_ref = pd.read_csv(data_path / 'cds_reference.csv')

In [6]:
# store = Store()
store.data['bond_ref'] = copy.deepcopy(get_bond_ref())
store.data['bond_ref_hist'] = get_bond_ref()
store.data['bond_hist'] = copy.deepcopy(bh)
store.data['cds_ref'] = copy.deepcopy(cds_ref)
store.bq = bq
# store.data['issuers_curves'] = copy.deepcopy(issuers_curves)

In [7]:
live = Tabs(
    {
        'curves': CurvesViewer(name='live_curves', build=False),
        'pivot': Pivot(build=False),
        'max diff': LowestFieldCurve(build=False),
        'visual': Visual(build=False),
        'clarion comp': Comp(build=False),
        'cdx': CDXComp(build=False),
        'grid': Grid(build=False)
    },
    store=store
)

In [8]:
hist = Tabs(
    {
        'curves': CurvesViewerH(name='hist_curves', build=False),
        'pivot': PivotH(curve_key='hist_curves', build=False),
        'max diff': LowestFieldCurveH(build=False),
        'visual': VisualH(build=False),
        'fwd pair': ForwardH(build=False),
        'fwd pivot': PivotForwardH(build=False),
        'hazard': HazardH(build=False)
    },
)

In [9]:
router.register(
    {
        'live': live.view,
        'hist': hist.view,
        'risk': v.Container(children=['Work in progress..'])
    }
)
# router.value = 'live'

In [10]:
drawer_items = v.List(
    nav=True,
    dense=True,
    children=[
        v.ListItem(
            link=True,
            tag='live',
            children=[
                v.ListItemIcon(children=[v.Icon(children=['mdi-alpha-l-box'])]),
                v.ListItemTitle(children='Live')
            ],
        ),
        v.ListItem(
            link=True,
            tag='hist',
            children=[
                v.ListItemIcon(children=[v.Icon(children=['mdi-alpha-h-box'])]),
                v.ListItemTitle(children='History')
            ]
        ),
        v.ListItem(
            link=True,
            tag='risk',
            children=[
                v.ListItemIcon(children=[v.Icon(children=['mdi-alpha-r-box'])]),
                v.ListItemTitle(children='Risk')
            ]
        ),
    ]
)

In [11]:
def change_view(
    widget, 
    event, 
    payload, 
    key, 
    drawer_items, 
    color='red'
):
    router.value = key
    for list_item in drawer_items.children:
        color_ = color if list_item is widget else None
        for child in list_item.children:
            try:
                if isinstance(child, v.ListItemIcon):
                    child.children[0].color = color_
            except:
                pass

In [12]:
for i, list_item in enumerate(drawer_items.children):
    list_item.on_event('click', partial(change_view, key=list_item.tag, drawer_items=drawer_items))
    if not i:
        list_item.fire_event('click', None)

In [13]:
drawer = v.NavigationDrawer(
    mini_variant=True,
    permanent=False,
    expand_on_hover=False,
    app=True,
    children=[drawer_items]
)

In [14]:
app = v.App(
    children=[
        drawer,
        v.Content(children=[v.Container(class_="ma-0 pa-0", fluid=True, children=[router.view])])
    ]   
)
app

App(children=[NavigationDrawer(app=True, children=[List(children=[ListItem(children=[ListItemIcon(children=[Ic…